In [1]:
import xmltodict
import math
import urllib.request
import itertools
import pandas as pd
import pickle   
from sklearn.utils import shuffle
import time
import numpy as np
import traceback
from IPython.display import display_html 


In [2]:
import sqlite3
from sqlite3 import Error

def create_con(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)

def create_table(conn, create_table_sql):
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
        conn.commit()
    except Error as e:
        print(e)

def create_game(conn, game):
    sql = """ INSERT INTO games(name)
              VALUES(?) """
    c = conn.cursor()
    c.execute(sql, game)
    conn.commit()
    return c.lastrowid


def create_reviews(conn, reviews):
    sql = """ INSERT INTO reviews(ID, user, rating, comment)
              VALUES(?,?,?,?) """
    cur = conn.cursor()
    for review in reviews:
        cur.execute(sql,review)
    cur.execute(sql, review)
    conn.commit()
    return cur.lastrowid

def do_sql(conn,sql):
    cur = conn.cursor()
    cur.execute(sql)
    rows = cur.fetchall()
    for row in rows:
        print(row)

def count_reviews(conn):
    cur = conn.cursor()
    print(cur.execute("SELECT COUNT (*) FROM reviews;").fetchall()[0])

def count_distinct_reviews(conn):
    cur = conn.cursor()
    print(cur.execute("SELECT COUNT(*) FROM (SELECT DISTINCT * FROM reviews);").fetchall()[0])

def del_all_records(conn):
    cur = conn.cursor()
    cur.execute("DROP TABLE IF EXISTS reviews")
    cur.execute("DROP TABLE IF EXISTS games")
    conn.commit()

In [3]:
from urllib.request import urlopen
# the original csv from https://raw.githubusercontent.com/beefsack/bgg-ranking-historicals/master/2017-02-21.csv
# The ID's where used in API calls to retrieve the game reviews
link="https://raw.githubusercontent.com/beefsack/bgg-ranking-historicals/master/2022-01-10.csv"
f = urlopen(link)
games = pd.read_csv(f)
games.describe()
games.sort_values('Users rated',ascending=False,inplace=True)
# games.set_index('ID', inplace=True)
games[:5]


,ID,Name,Year,Rank,Average,Bayes average,Users rated,URL,Thumbnail
105,30549,Pandemic,2008,106,7.59,7.487,109038,/boardgame/30549/pandemic,https://cf.geekdo-images.com/S3ybV1LAp-8SnHIXL...
190,822,Carcassonne,2000,191,7.42,7.308,108816,/boardgame/822/carcassonne,https://cf.geekdo-images.com/okM0dq_bEXnbyQTOv...
429,13,Catan,1995,430,7.14,6.970,108108,/boardgame/13/catan,https://cf.geekdo-images.com/W3Bsga_uLP9kO91gZ...
72,68448,7 Wonders,2010,73,7.74,7.634,90050,/boardgame/68448/7-wonders,https://cf.geekdo-images.com/RvFVTEpnbb4NM7k0I...
103,36218,Dominion,2008,104,7.61,7.499,81600,/boardgame/36218/dominion,https://cf.geekdo-images.com/j6iQpZ4XkemZP07HN...


In [4]:
from sqlalchemy import create_engine

sql_create_table_reviews = """ CREATE TABLE IF NOT EXISTS reviews(
                                    review_id integer PRIMARY KEY,
                                    ID integer,
                                    user text NOT NULL,
                                    rating NOT NULL,
                                    comment text,              
                                    FOREIGN KEY(ID) REFERENCES games(ID));"""

conn = create_con('bgg2022.db')
# del_all_records(conn) # empty database (optional)

if conn:
    engine = create_engine('sqlite:///%s' % 'bgg2022.db', echo=True)
    games[['ID','Name']].to_sql('games', con=engine)
    # c = conn.cursor()
    # c.execute(""" DROP TABLE IF EXISTS games""")
    # create_table(conn, sql_create_table_games)
    # c.execute(""" DROP TABLE IF EXISTS reviews""")
    create_table(conn, sql_create_table_reviews)
    count_reviews(conn)


2022-01-12 15:17:30,490 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("games")
2022-01-12 15:17:30,491 INFO sqlalchemy.engine.Engine [raw sql] ()


ValueError: Table 'games' already exists.

In [5]:
def gen_batches():
    num_batches = len(games)//100
    for b in range(num_batches):
        result = slice(b*100,(b+1)*100)
        if b*100>-1:
            yield result
    yield slice((b+1)*100,len(games)+1)
    
def number_rating_pages(game_id):
    url = 'https://www.boardgamegeek.com/xmlapi2/thing?id='+str(game_id)+'&ratingcomments=1'
    u = urllib.request.urlopen(url).read()#The url you want to open
    doc = xmltodict.parse(u)
    return math.ceil(int(doc['items']['item']['comments']['@totalitems'])/100)

def get_url_result(ids,p): # given a url of an object and a page, return list of dicts with the comments
    url = 'https://www.boardgamegeek.com/xmlapi2/thing?id='+ids+'&ratingcomments=1'
    url += '&&page='+str(p)
    u = urllib.request.urlopen(url).read()#The url you want to open
    doc = xmltodict.parse(u)
    return doc 

def process_result(url_result):
    remove_list = []
    for g in url_result['items']['item']:
        if not isinstance(g, str):
            if 'comment' in g['comments'].keys():
                try:
                    review_tup = [(g['@id'],) + tuple(review.values()) for review in g['comments']['comment']]
                    # 'str' object has no attribute 'values' Traceback (most recent call last):
                    create_reviews(conn,review_tup)
                except Exception as e:
                    print(e,traceback.format_exc())
            else:
                remove_list.append(int(g['@id']))
        else:
            print('crazy str',g)
    return remove_list


In [110]:
conn = create_con('test.db')
game_generator = gen_batches()

# get all games, but do this per batch of 100 games
for batch in game_generator: 
    print('new batch,', batch)
    # list of games
    games_in_batch = list(games['ID'][batch])
    
    #get maximum number of pages from game on top of the batch
    num_pages = number_rating_pages(games_in_batch[0])+1
    if num_pages>0:
        # get all the pages, quering as little games as possible and store the results
        for p in range(0,num_pages):
            for tryout in range(5):
                try:
                    print('next query for page {} for {} games'.format(p,len(games_in_batch)))
                    #print('games still in batch are \n',games_in_batch)
                    if not games_in_batch:
                        print('gamelist empty!')
                        break
                    # convert list of current games to url input
                    ids = ','.join(str(g) for g in games_in_batch)
                    #query API and parse to dictionary
                    url_result = get_url_result(ids,p)
                    print('data retrieved')
                    # remove games that have no comment, they dont have to be queried again
                    removelist = process_result(url_result)
                    #print('to remove',removelist)
                    games_in_batch = [g for g in games_in_batch if g not in removelist]
                    time.sleep(8)
                    # if p%1000==0:
                    #     with open('ratings.pickle', 'wb') as handle:
                    #         pickle.dump(rating_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
                    break
                except Exception as e:
                    print('error',tryout,p,e)
                    time.sleep(30)
print('all done')
conn.close()

next query for page 1 for 100 games
data retrieved
next query for page 2 for 100 games
data retrieved


## Preprocessing

In [25]:
from database import DB
%load_ext autoreload
%autoreload 2

In [26]:
db = DB('bgg2022.db')
db.describe('reviews')

[('games', 21839), ('reviews', 18964823)]
('reviews', 18964823)


,review_id,ID,user,rating,comment
0,10101,30549,Torsten,10,
1,10102,30549,mitnachtKAUBO-I,10,Hands down my favorite new game of BGG CON 200...
2,10103,30549,avlawn,10,I tend to either love or easily tire of co-op ...
3,10104,30549,Mike Mayer,10,
4,10105,30549,Mease19,10,This is an amazing co-op game. I play mostly ...


In [40]:
sql = """COMMIT;
"""
db(sql)

[]

In [4]:
# Check if all game ID's are matching 
sql = """
SELECT ID FROM reviews
WHERE ID NOT IN (SELECT ID FROM games) """
print(len(db(sql)), 'rows with nonmatching ID')

sql = """
DELETE FROM reviews 
WHERE ID NOT IN (SELECT ID FROM games) """
db(sql)

0 rows with nonmatching ID


[]

In [5]:
sql = """   SELECT ID, user, COUNT(*)
            FROM reviews 
            GROUP BY ID, user
            HAVING COUNT(*) > 1"""
res = db(sql)

In [6]:
len(res)

2029653

In [7]:
# certain users have many reviews for the same game in the db. Often, the score is given for a variant
for gameid, user, times in res:
    if times > 70:
        sql = f"   SELECT g.name, r.user FROM reviews r  JOIN games g ON g.ID = r.ID WHERE r.ID = {gameid} AND r.user = '{user}' GROUP BY 1,2"      
        game, user = db(sql)[0]
        print(f'user {user} reviewed {game} ({gameid}) {times} times')

user pcstam reviewed Chess (171) 107 times
user pcstam reviewed Dominoes (2394) 84 times
user pcstam reviewed Backgammon (2397) 79 times
user loopoocat reviewed Top Trumps (7262) 86 times
user loopoocat reviewed Old Maid (10799) 201 times
user loopoocat reviewed Snap (17003) 89 times
user GrubyOi reviewed Warhammer 40,000: Codex (39169) 100 times
user loopoocat reviewed Quartett (133425) 192 times


In [8]:
# example
db.df("SELECT * FROM reviews WHERE user = 'loopoocat' and ID = 10799 GROUP BY rating, comment LIMIT 10")

,review_id,ID,user,rating,comment
0,15135942,10799,loopoocat,2,
1,15085514,10799,loopoocat,3,
2,15045069,10799,loopoocat,4,
3,15014787,10799,loopoocat,5,
4,15004626,10799,loopoocat,6,
5,14984459,10799,loopoocat,7,


In [9]:
sql = """
SELECT * FROM reviews
    WHERE review_id NOT IN
    (
        SELECT MAX(review_id) AS MaxRecordID
        FROM reviews
        GROUP BY ID, user
    );
"""
len(db(sql))

2107129

The amount of duplicates is around 2 million

In [10]:
sql = """
DELETE FROM reviews
    WHERE review_id NOT IN
    (
        SELECT MAX(review_id) AS MaxRecordID
        FROM reviews
        GROUP BY ID, user
    );
"""
db(sql)

[]

In [12]:
db.commit()

In [8]:
db.count()

NameError: name 'db' is not defined

In [18]:
conn = create_con('bgg2022.db')
do_sql(conn, "SELECT COUNT(*) FROM reviews")

(18964823,)


In [22]:
for val in ['user','rating','ID']:
    print(val,conn.execute("SELECT COUNT(*) FROM (SELECT DISTINCT "+ val +" FROM reviews);").fetchall())

user [(412831,)]
rating [(10772,)]
ID [(21839,)]


# Exporting DB to CSV

In [41]:
import pandas as pd
import sqlite3
from database import DB

# Create your connection.
conn = DB('bgg2022.db').conn

df = pd.read_sql_query("SELECT * FROM reviews", conn)

[('games', 21839), ('reviews', 18964807)]


In [42]:
df['ID'] = df['ID'].astype('int64')

In [43]:
df = df.merge(games[['ID','Name']],on='ID',how='left')

In [44]:
df.head()

,review_id,ID,user,rating,comment,Name
0,10101,30549,Torsten,10,,Pandemic
1,10102,30549,mitnachtKAUBO-I,10,Hands down my favorite new game of BGG CON 200...,Pandemic
2,10103,30549,avlawn,10,I tend to either love or easily tire of co-op ...,Pandemic
3,10104,30549,Mike Mayer,10,,Pandemic
4,10105,30549,Mease19,10,This is an amazing co-op game. I play mostly ...,Pandemic


In [45]:
len(df)

18964807

In [46]:
df.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=True)

In [47]:
len(df)

18964807

In [48]:
df.drop_duplicates(subset=['user','ID'], keep='first', inplace=True, ignore_index=True)

In [49]:
len(df)

18964807

In [50]:
df.rename(columns={'Name':'name'},inplace=True)
df = df[['user', 'rating', 'comment', 'ID', 'name']]

412816

0           10.0
1           10.0
2           10.0
3           10.0
4           10.0
            ... 
18964802     3.1
18964803     2.0
18964804     2.0
18964805     1.0
18964806     1.0
Name: rating, Length: 18964807, dtype: float64

In [71]:
# with .loc to don't get the copy warning
df.loc[:,["rating"]] = pd.to_numeric(df["rating"])
df.loc[:,["ID"]] = pd.to_numeric(df["ID"])
df.loc[:,['name']] = pd.Categorical(df['name'])

In [72]:
filename = 'bgg-19m-reviews.csv'
df.to_csv(filename)

### Importing to check if ok

In [73]:
import pandas as pd
import sqlite3
reviews = pd.read_csv('bgg-19m-reviews.csv',index_col=0)
reviews.head()

In [74]:
reviews.head()

,user,rating,comment,ID,name
0,Torsten,10.0,NaN,30549,Pandemic
1,mitnachtKAUBO-I,10.0,Hands down my favorite new game of BGG CON 200...,30549,Pandemic
2,avlawn,10.0,I tend to either love or easily tire of co-op ...,30549,Pandemic
3,Mike Mayer,10.0,NaN,30549,Pandemic
4,Mease19,10.0,This is an amazing co-op game. I play mostly ...,30549,Pandemic
